In [3]:
from Model.model import ModelSTGCN
import torch
from torch.utils.data import DataLoader,Dataset
import pandas as pds
import numpy as np
import torch.nn as nn

c:\Users\PhuTuan\anaconda3\envs\pythonProject\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
class ActionDataset(Dataset):
    def __init__(self,Transform=None):
        super().__init__()
        # assert all(param is not None for param in [Data,label]),"Data and label must be give in"
        self.transform = Transform
        self.lst=[]
        self.label=[]
        # self.append(Data,label)

    def __getitem__(self, index):
        sample = self.lst[index],self.label[index]
        if self.transform:
            sample=self.transform(sample)
        return sample
    
    def __len__(self):
        return self.leng
    
    def SetTrans(self,Transform):
        self.transform=Transform
    
    def append(self,Data):
        # super().__init__()
        # assert all(param is not None for param in [Data,label]),"Data and label must be give in"
        # assert label==None,"Label must be give"
        kpscore = np.expand_dims(Data['keypoint_score'][0],axis=2)
        kp = Data['keypoint'][0]
        data = np.concatenate((kp,kpscore),axis=2)
        # data = np.expand_dims(data,axis=0)
        label = Data['label']
        self.lst.append(data)
        self.label.append(label)
        self.leng=len(self.lst)  

class ToTensor():
    def __call__(self,sample):
        data,label=sample
        return torch.from_numpy(data.astype(np.float32)),torch.tensor(label)

In [5]:
# a=torch.rand(2,1).argmax(dim=0)
# val =a.item()

In [6]:
# torch.__version__

In [7]:
def Train_epoch(model=None,loss_fn=None,train_loader=None,optimizer=None,device='cuda'):
    device=torch.device(device if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.train()
    # model.training = True
    total_loss=0
    for index,(data,label) in enumerate(train_loader):
        outputs=model(data.to(device))
        label=label.to(device)
        loss = loss_fn(outputs,label)
        total_loss+=loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    total_loss/=(index+1)
    return total_loss    

def Val_epoch(model=None,loss_fn=None,val_loader=None,device='cuda'):
    device=torch.device(device if torch.cuda.is_available() else 'cpu')
    model.eval()
    model.to(device)
    total_loss = 0
    preds = []
    labels = []
    model.training = True
    with torch.no_grad():
        for index,(data,label) in enumerate(val_loader):
            outputs = model(data.to(device))
            label = label.to(device)
            loss = loss_fn(outputs,label)
            pred = torch.argmax(outputs,dim=1)
            labels.append(label.item())
            preds.append(pred.item())
            total_loss+=loss
    total_loss/=(index + 1)
    #acc multi class CrossEntropy
    acc = eval_acc(preds,labels)
    return total_loss,acc

def eval_acc(preds,labels):
    n_total = len(preds)
    n_correct = 0
    for pred,label in zip(preds,labels):
        if pred == label: n_correct+=1
        else: continue
    acc=n_correct/n_total
    return acc
        

In [8]:
def Train_n_Eval(model=None,epochs=None,loss_fn=None,lr=1e-4,
                 train_dataloader=None,eval_dataloader=None,lr_shedule=False,
                 Step=10,miles=2,Gamma=0.1):
    assert all(param is not None for param in [model,epochs,loss_fn,
                                               train_dataloader,eval_dataloader]),"All Param must be give in"
    optim = torch.optim.Adam(model.parameter(),lr)
    loss_history = {
        'train': [],
        'val' : [],
    }
    if lr_shedule:
        Multistep=[Step * i for i in range(1,miles+1)]
        sheduler = torch.optim.lr_scheduler.MultiStepLR(optim,Multistep,Gamma)
    for epoch in range(epochs):
        train_loss = Train_epoch(model,loss_fn,train_dataloader,optim)
        val_loss,acc = Val_epoch(model,loss_fn,eval_dataloader)
        loss_history['train'].append(train_loss)
        loss_history['val'].append(val_loss)
        if lr_shedule:
            sheduler.step()
        print(f'Epoch: {epoch}:\n \tTrain Loss: {train_loss}\n\tVal Loss: {val_loss}, Accuracy: {acc}')
    torch.save(model.state_dict(),'Checkpoint/model_final.pth')
        

In [9]:
model=ModelSTGCN(3,2)
criterion=nn.CrossEntropyLoss()
datalst1 = pds.read_pickle('Data/train.pkl')
datalst2 = pds.read_pickle('Data/val.pkl')
train_dataset = ActionDataset(ToTensor())
val_dataset = ActionDataset(ToTensor())
for data in datalst1:
    train_dataset.append(data)
for data in datalst2:
    val_dataset.append(data)


In [10]:
train_dataloader=DataLoader(dataset=train_dataset,shuffle=True)
val_dataloader=DataLoader(dataset=val_dataset,shuffle=True)

In [11]:
dataview=iter(train_dataloader)
data = dataview.next()

In [14]:
data[0].size()

torch.Size([1, 128, 17, 3])

In [ ]:
# def Train_val(model=None,dataloader=None,epochs=None,lr=1e-4,lr_schedule=None):
#     assert all(param is not None for param in [model,dataloader,epochs]),"All param must be give in: model,dataloader,epochs"
#     device=torch.device('cuda')
#     model.to(device)
#     optim=torch.optim.Adam(model.parameters(),lr)
#     criteria=nn.CrossEntropyLoss()
#     # criteria=nn.BCELoss()
#     for epoch in range(epochs):
#         for i,(data,label) in enumerate (dataloader):
#             output=model(data.to(device))
#             label=label.to(device)
#             # _,predict=torch.max(output,dim=1)
#             loss = criteria(output,label)
#             optim.zero_grad()
#             loss.backward()
#             optim.step()
#             print(f'Epoch: {epoch}, step: {i}/{len(dataloader)}, Loss: {loss:.5f}')
    

In [ ]:
# datalst1 = pds.read_pickle('Data/train.pkl')
# datalst2 = pds.read_pickle('Data/val.pkl')
# train_dataset = ActionDataset(ToTensor())
# val_dataset = ActionDataset(ToTensor())
# for data in datalst1:
#     train_dataset.append(data)
# for data in datalst2:
#     val_dataset.append(data)


In [ ]:
# # for epoch in range(2):
# #     for i,data in enumerate (dataloader):
# #         print(f'Epoch: {epoch}, Step: {i+1}/{len(dataloader)}, label: {data[1]}')
# dataview=iter(dataloader)
# data = dataview.next()

In [ ]:
testfile=pds.read_pickle('test.pkl')

In [ ]:
testact=ActionDataset(ToTensor())
testact.append(testfile)

In [ ]:
torch.expand_copy()

In [ ]:
# batch_size=4
# datalst=pds.read_pickle('train.pkl')
# actDataset=ActionDataset(datalst[0])
# dataloader=DataLoader(dataset=actDataset,batch_size=batch_size,shuffle=True)
# n_sample=len(actDataset)
# epochs=10
# n_iter=round(n_sample/batch_size)
# for epoch in range(epochs-8):
#     for i,data in enumerate (dataloader):
#         if (i+1)% (batch_size)==0:
#             print(f'Epoch: {epoch}, Step: {i+1}/{n_iter}, Input shap: {data.shape}')

In [ ]:
# import math
# math.ceil(len(actDataset)/batch_size)

In [ ]:
# batch_size=4
# datalst=pds.read_pickle('train.pkl')
# actDataset=ActionDataset(datalst[0])
# dataloader=DataLoader(dataset=actDataset,batch_size=batch_size,shuffle=True)
# n_sample=len(actDataset)
# n_iter=round(n_sample/batch_size)
# model = ModelSTGCN(3,5)
# lr = 1e-3
# optim = torch.optim.Adam(model.parameters(),lr)
# lr_schedule = torch.optim.lr_scheduler.MultiStepLR(optim,[15,30],gamma=0.1)
# epochs = 10
# for epoch in range(epochs):
#     for i,data in enumerate (dataloader):
#         if (i+1)% (batch_size)==0:
#             print(f'Epoch: {epoch}, Step: {i+1}/{n_iter}')